In [1]:
import sys
sys.path.append("../src")
sys.path.append("../src/models")

from models.duplo import DuPLO
import torch
from train import prepare_dataset
from argparse import Namespace
from tqdm.notebook import tqdm
import numpy as np
import sklearn.metrics
import pandas as pd
import os
from datasets.BavarianCrops_Dataset import BavarianCropsDataset
from train_duplo import metrics

model_root = "../models/duplo"

def merge(namespaces):
    merged = dict()

    for n in namespaces:
        d = n.__dict__
        for k,v in d.items():
            merged[k]=v

    return Namespace(**merged)

TUM_dataset = Namespace(
    dataset = "BavarianCrops",
    trainregions = ["holl","nowa","krum"],
    testregions = ["holl","nowa","krum"],
    scheme="blocks",
    test_on = "test",
    train_on = "trainvalid",
    samplet = 70
)

GAF_dataset = Namespace(
    dataset = "GAFv2",
    trainregions = ["holl","nowa","krum"],
    testregions = ["holl","nowa","krum"],
    features = "optical",
    scheme="blocks",
    test_on="test",
    train_on="train",
    samplet = 23
)

def setup(dataset, mode, dataroot="../data", store = '/tmp/'):
    
    if mode == "12classes":
        classmapping = os.path.join(dataroot,"BavarianCrops",'classmapping12.csv')
    elif mode == "23classes":
        classmapping = os.path.join(dataroot,"BavarianCrops",'classmapping23.csv')
    
    args = Namespace(batchsize=256,
                 classmapping=classmapping,
                 dataroot=dataroot, dataset=dataset,
                 model='duplo',mode=None,
                 seed=0, store=store, workers=0)

    if dataset == "BavarianCrops":
        args = merge([args,TUM_dataset])
        exp = "isprs_tum_duplo"
    elif dataset == "GAFv2":
        args = merge([args,GAF_dataset])
        exp = "isprs_gaf_duplo"
        
    traindataloader, testdataloader = prepare_dataset(args)
        
    input_dim = traindataloader.dataset.datasets[0].ndims
    nclasses = len(traindataloader.dataset.datasets[0].classes)

    device = torch.device("cuda")
    model = DuPLO(input_dim=input_dim, nclasses=nclasses, sequencelength=args.samplet, dropout=0.4)
    model.load(f"{model_root}/{mode}/{exp}/model.pth")

    model.to(device)

    
    return testdataloader, model

def evaluate(model, dataloader):
    stats = list()
    model.cuda()
    model.eval()
    y_preds = list()
    ys = list()
    ids = list()
    with torch.no_grad():
        for idx, batch in tqdm(enumerate(dataloader),total=len(dataloader)):
            X,y,id = batch
            logsoftmax , *_ = model.forward(X.transpose(1,2).cuda())
            y_pred = logsoftmax.argmax(dim=1)
            ys.append(y.cpu().detach().numpy())
            y_preds.append(y_pred.cpu().detach().numpy())
            ids.append(id)
    model.cpu()
    return np.hstack(y_preds), np.vstack(ys)[:,0], np.hstack(ids)
    

In [2]:
testdataloader, model = setup("BavarianCrops","23classes")
y_pred, y, ids = evaluate(model, testdataloader)
print(sklearn.metrics.classification_report(y,y_pred))
metrics(y,y_pred)

Initializing BavarianCropsDataset test partition in holl
read 23 classes
precached dataset files found at ../data/BavarianCrops/npy/classmapping23.csv/blocks/holl/test
loaded 9792 samples
Dataset ../data/BavarianCrops. region holl. partition test. X:9792x(144, 13), y:(9792,) with 23 classes
Initializing BavarianCropsDataset test partition in nowa
read 23 classes
precached dataset files found at ../data/BavarianCrops/npy/classmapping23.csv/blocks/nowa/test
loaded 3572 samples
Dataset ../data/BavarianCrops. region nowa. partition test. X:3572x(287, 13), y:(3572,) with 23 classes
Initializing BavarianCropsDataset test partition in krum
read 23 classes
precached dataset files found at ../data/BavarianCrops/npy/classmapping23.csv/blocks/krum/test
loaded 4306 samples
Dataset ../data/BavarianCrops. region krum. partition test. X:4306x(143, 13), y:(4306,) with 23 classes
Initializing BavarianCropsDataset trainvalid partition in holl
read 23 classes
precached dataset files found at ../data/Bava

/home/tanjara/.conda/envs/crop-type-mapping/lib/python3.7/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


  0%|          | 0/70 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.19      0.07      0.10       356
           1       0.46      0.68      0.55       225
           2       0.65      0.29      0.40       882
           3       0.00      0.00      0.00       469
           4       0.00      0.00      0.00        23
           5       0.65      0.78      0.71       995
           6       0.92      0.96      0.94      1917
           7       0.00      0.00      0.00        56
           8       0.50      0.06      0.11        16
           9       0.38      0.08      0.14       121
          10       0.84      0.97      0.90      9301
          11       0.25      0.00      0.01       226
          12       0.00      0.00      0.00        52
          13       0.27      0.12      0.17       104
          14       0.60      0.84      0.70       777
          15       0.74      0.82      0.78      1017
          16       0.53      0.38      0.44       248
          17       0.46    

/home/tanjara/.conda/envs/crop-type-mapping/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tanjara/.conda/envs/crop-type-mapping/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tanjara/.conda/envs/crop-type-mapping/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

{'accuracy': 0.7900396151669496,
 'kappa': 0.6795626476440702,
 'f1_micro': 0.7900396151669495,
 'f1_macro': 0.342844877582905,
 'f1_weighted': 0.747589372509714,
 'recall_micro': 0.7900396151669496,
 'recall_macro': 0.3417897309311548,
 'recall_weighted': 0.7900396151669496,
 'precision_micro': 0.7900396151669496,
 'precision_macro': 0.4343676608558107,
 'precision_weighted': 0.73517078259069}

In [3]:
#test = torch.ones([ 256, 70, 13], dtype=torch.FloatTensor, device='cuda:0')
test = torch.ones(1, 70, 13)
logsoftmax , *_ = model.forward(test.transpose(1,2))
y_pred = logsoftmax.argmax(dim=1)

In [4]:
print(y_pred)

tensor([9])


In [48]:
crop_csv = pd.read_csv('../data/Test/random_crop1.csv')
droped_csv = crop_csv.drop({'lat', 'lon'},axis = 1)
droped_csv = droped_csv.iloc[0:70, :]
print(droped_csv.head)
tensor_crop = torch.tensor(droped_csv.values).unsqueeze(0)
print(tensor_crop.shape)

<bound method NDFrame.head of      Band1   Band2   Band3   Band4   Band5   Band6   Band7   Band8   Band9  \
0   1170.0   935.0   934.0   725.0  1131.0  2606.0  3584.0  3643.0  3832.0   
1   1170.0   938.0   965.0   775.0  1131.0  2606.0  3584.0  3673.0  3832.0   
2   1170.0   963.0  1000.0   796.0  1243.0  2700.0  3523.0  3509.0  3825.0   
3   1170.0   979.0  1021.0   847.0  1243.0  2700.0  3523.0  3394.0  3825.0   
4   1205.0   989.0  1028.0   879.0  1246.0  2642.0  3458.0  3319.0  3763.0   
5   1205.0   989.0  1028.0   879.0  1246.0  2642.0  3458.0  3319.0  3763.0   
6   1205.0   982.0  1006.0   873.0  1246.0  2642.0  3458.0  3350.0  3763.0   
7   1205.0   961.0  1014.0   850.0  1209.0  2615.0  3418.0  3431.0  3749.0   
8   1170.0   937.0   958.0   759.0  1154.0  2549.0  3451.0  3524.0  3793.0   
9   1170.0   941.0   960.0   785.0  1154.0  2549.0  3451.0  3435.0  3793.0   
10  1170.0   977.0   971.0   814.0  1261.0  2546.0  3293.0  3358.0  3653.0   
11  1170.0  1004.0  1042.0   952.0

In [40]:
logsoftmax , *_ = model.forward(tensor_crop.float().transpose(1,2))
y_pred = logsoftmax.argmax(dim=1)
print(y_pred)

tensor([15])


In [24]:
#RNN MODEL
import sys
import os


sys.path.append("../src")
sys.path.append("../src/models")

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import numpy as np
from rnn import RNN
import torch
import torch.nn.functional as F
import tqdm
from torch import matmul, sigmoid, tanh

sns.set_style("whitegrid")

BANDS = ['B1', 'B10', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8',
       'B8A', 'B9']

# simulate no GPU (for checks like torch.cuda.is_available())
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [44]:
modelRNN = RNN(input_dim=13, nclasses=33, hidden_dims=32,
                      num_rnn_layers=4, dropout=0.710883, bidirectional=True)
modelRNN.load("../data/notebookdata/rnn.pth")
modelRNN.eval()

loading model from ../data/notebookdata/rnn.pth


RNN(
  (inlayernorm): LayerNorm((13,), eps=1e-05, elementwise_affine=True)
  (clayernorm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (lstm): LSTM(13, 32, num_layers=4, bias=False, batch_first=True, dropout=0.710883, bidirectional=True)
  (linear_class): Linear(in_features=256, out_features=33, bias=True)
)

In [49]:
logsoftmax , *_ = modelRNN.forward(tensor_crop.float().transpose(1,2))
y_pred = logsoftmax.argmax(dim=1)
print(y_pred)

tensor([22])
